# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [22]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
# add the following codes 
# in case show the error "ProgrammingError: can't adapt type 'numpy.int64'"
# import numpy
# from psycopg2.extensions import register_adapter, AsIs
# def addapt_numpy_float64(numpy_float64):
#     return AsIs(numpy_float64)
# def addapt_numpy_int64(numpy_int64):
#     return AsIs(numpy_int64)
# register_adapter(numpy.float64, addapt_numpy_float64)
# register_adapter(numpy.int64, addapt_numpy_int64)

In [23]:
# run create_tables.py 
# %run create_tables.py 

In [24]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [19]:
filepath = "data/song_data"

In [20]:
song_files = get_files(filepath)

In [8]:
# song_files

In [21]:
df = pd.concat((pd.read_json(file, lines=True) for file in song_files), ignore_index=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
2,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
4,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


In [10]:
# df.dtypes

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].drop_duplicates().values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, (song_data[0], song_data[1], song_data[2], song_data[3], song_data[4]))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
#artist_data = artist_data
artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
2,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892


In [14]:
# len(artist_data["artist_id"])

In [15]:
# len(artist_data["artist_id"].drop_duplicates())

In [16]:
# len(artist_data)

In [17]:
# len(artist_data.drop_duplicates())

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [18]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

KeyError: 0

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
filepath = "data/log_data"

In [ ]:
log_files = get_files(filepath)

In [ ]:
df = pd.concat((pd.read_json(file, lines=True) for file in log_files), ignore_index=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
# pd.to_datetime(df["ts"], unit = "ms")

In [ ]:
# t = pd.to_datetime(df["ts"], unit="ms")

In [ ]:
df = df.loc[df["page"] == "NextSong", ]
df["ts"] = pd.to_datetime(df["ts"], unit = "ms")
df.head()

In [ ]:
# t.dt.hour.values

In [ ]:
t = df
t.head()

In [ ]:
time_data = (df["ts"].values, # timestamp
            df["ts"].dt.hour.values, # hour
            df["ts"].dt.day.values,  # day
            df["ts"].dt.week.values, # week of the year
            df["ts"].dt.month.values, # month
            df["ts"].dt.year.values,  # year 
            df["ts"].dt.weekday.values) # weekday
column_labels = ("timestamp", "hour", "day", "week_of_year", "month", "year", "weekday")

In [ ]:
# create time data dictionary
time_dictionary = {}
for i in range(7):
    time_dictionary[column_labels[i]] = time_data[i]

In [ ]:
time_df = pd.DataFrame(time_dictionary)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]

In [ ]:
user_df.dtypes

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (songid, artistid, row.ts, row.userId, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.